In [14]:
import os
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import datetime
import numpy as np
import yfinance as yf

In [3]:
load_dotenv()

True

In [4]:
# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(
                alpaca_api_key,
                alpaca_secret_key,
                api_version="v2")

In [15]:
tickers = ["SPY", "RPAR", "VBAIX"]

# Set timeframe to 1D 
timeframe="1D"

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
# Alternatively you can use a start and end date of 2020-08-07
todays_date = str(datetime.datetime.now()).split()[0]
one_year_ago = str(datetime.datetime.now() - datetime.timedelta(days=365)).split()[0]

start_date = pd.Timestamp(one_year_ago, tz='America/New_York').isoformat()
end_date = pd.Timestamp(todays_date, tz='America/New_York').isoformat()

In [16]:
current_closing_prices_df = alpaca.get_barset(
                                        tickers,
                                        timeframe,
                                        start=start_date,
                                        end=end_date).df

# Review the first 5 rows of the Alpaca DataFrame
current_closing_prices_df.head()

RPAR                                      SPY  \
                            open    high      low   close  volume    open   
time                                                                        
2021-03-09 05:00:00+00:00  22.86  23.040  22.8600  23.005  176733  385.85   
2021-03-10 05:00:00+00:00  23.06  23.125  22.9800  23.110  113421  389.69   
2021-03-11 05:00:00+00:00  23.19  23.250  23.1101  23.210  121728  392.23   
2021-03-12 05:00:00+00:00  22.98  22.980  22.8250  22.925   92146  392.07   
2021-03-15 04:00:00+00:00  22.97  23.060  22.8901  23.050  288903  394.33   

                                                                  VBAIX       \
                              high       low     close     volume  open high   
time                                                                           
2021-03-09 05:00:00+00:00  389.910  381.7299  387.1100   97898909   NaN  NaN   
2021-03-10 05:00:00+00:00  391.400  388.1702  389.5199  101349197   NaN  NaN   
2021-03-11 05:00:00+00:00  395.650  391.7400  393.5000   77022138   NaN  NaN   
2021-03-12 05:00:00+00:00  394.210  391.2000  394.0900   56997361   NaN  NaN   
2021-03-15 04:00:00+00:00  396.685  392.0300  396.4700   64471980   NaN  NaN   

                                             
                           low close volume  
time                                         
2021-03-09 05:00:00+00:00  NaN   NaN    NaN  
2021-03-10 05:00:00+00:00  NaN   NaN    NaN  
2021-03-11 05:00:00+00:00  NaN   NaN    NaN  
2021-03-12 05:00:00+00:00  NaN   NaN    NaN  
2021-03-15 04:00:00+00:00  NaN   NaN    NaN

In [1]:
def sharpe_ratio(df, risk_free=0, periodicity=252):
    # convert anuualized risk free rate into appropriate value
    risk_free = (1+risk_free)**(1/periodicity)-1
    excess_return = df.mean() - risk_free
    calculated_sharpe = (excess_return/df.std())*np.sqrt(periodicity)
    return calculated_sharpe

In [52]:
def retrieve_yahoo_data(ticker = 'spy', start_date = '2007-07-01', end_date = '2020-12-31'):
    try:
        yahoo_data = yf.Ticker(ticker)
        print(f"Ticker is {ticker}")
        price_df = yahoo_data.history(start=start_date, end=end_date).Close.pct_change()
        price_df.name = ticker
        if price_df.shape[0] == 0:
            raise Exception("No Prices.")
        return price_df
    except Exception as ex:
        print(f"Sorry, Data not available for '{ticker}': Exception is {ex}")

In [58]:
def target_downside_deviation(df, minimum_acceptable_return = 0, periodicity=252):
    df_diff = df - minimum_acceptable_return
    df_positive_excess_return = np.where(df_diff < 0, df_diff, 0)
    calculated_target_downside_deviation = np.sqrt(np.nanmean(df_positive_excess_return ** 2))
    return calculated_target_downside_deviation

In [60]:
def sortino_ratio(df, risk_free = 0, periodicity = 252, include_risk_free_in_volatility = False):
    risk_free = (1 + risk_free) ** (1/periodicity) - 1
    df_mean = np.nanmean(df) - risk_free
    if include_risk_free_in_volatility == True:
        minimum_acceptable_return = risk_free
    else:
        minimum_acceptable_return = 0
    calculated_target_downside_deviation = target_downside_deviation(df,
                                                                     minimum_acceptable_return = minimum_acceptable_return)
    df_sortino = (df_mean/calculated_target_downside_deviation) * np.sqrt(periodicity)
    return df_sortino

In [63]:
def annualized_return(df, periodicity = 252):
    difference_in_years = len(df)/periodicity
    start_net_asset_value = 1.0
    cumprod_return = np.nancumprod(df + start_net_asset_value)
    end_net_asset_value = cumprod_return[-1]
    annual_return = end_net_asset_value ** (1 / difference_in_years) - 1
    return annual_return

In [72]:
def get_max_draw_down(df, return_data = False):
    start_net_asset_value = 1.0
    cumprod_return = np.nancumprod(df + start_net_asset_value)
    peak_return = np.maximum.accumulate(cumprod_return)
    draw_down = (cumprod_return - peak_return) / peak_return
    if return_data == True:
        data = draw_down
    else:
        data = np.abs(np.nanmin(draw_down))
    return data

In [68]:
def return_max_drawdown_ratio(df, risk_free = 0, periodicity = 252):
    """df - asset return series, e.g. returns based on daily close prices of asset
   risk_free - annualized risk free rate (default is assumed to be 0)
   periodicity - number of periods at desired frequency in one year
                e.g. 252 business days in 1 year (default),
                12 months in 1 year,
                52 weeks in 1 year etc."""
    # convert annualized risk free rate into appropriate value for provided frequency of asset return series (df)
    risk_free= (1 + risk_free)**(1 / periodicity) - 1
    # determine annualized return to be used in numerator of return to max drawdown (RMDD) calculation
    annual_return = annualized_return(df, periodicity = periodicity)
    # determine max drawdown to be used in the denominator of RMDD calculation
    max_draw_down = get_max_draw_down(df, return_data = False)
    return (annual_return - risk_free) / abs(max_draw_down)

In [47]:
def average_positive(ret, drop_zero = 1):
    if drop_zero > 0:
        positives = ret > 0
    else:
        positives = ret >= 0
    if positives.any():
        return np.mean(ret[positives])
    else:
        return 0.000000000000000000000000000001

In [48]:
def average_negative(ret):
    negatives = ret < 0
    if negatives.any():
        return np.mean(ret[negatives])
    else:
        return -1*0.000000000000000000000000000001

In [76]:
def win_above_replace_port(
                    new_asset,
                    replace_port,
                    risk_free_rate = 0,
                    financing_rate = 0,
                    weight_asset = 0.25,
                    weight_replace_port = 1,
                    periodicity = 252):
    """Cole Win Above Replacement Portolio (CWARP): Total score to evaluate whether any new investment improves or hurts the return to risk of your total portfolio.
    new_asset = returns of the asset you are thinking of adding to your portfolio
    replace_port = returns of your pre-existing portfolio (e.g. S&P 500 Index, 60/40 Stock-Bond Portfolio)
    risk_free_rate = Tbill rate (annualized)
    financing_rate = portfolio margin/borrowing cost (annualized) to layer new asset on top of prevailing portfolio (e.g. LIBOR + 60bps). No financing rate is reasonable for derivate overlay products.
    weight_asset = % weight you wish to overlay for the new asset on top of the previous portfolio, 25% overlay allocation is standard
    weight_replace_port = % weight of the replacement portfolio, 100% pre-existing portfolio value is standard
    periodicity = the frequency of the data you are sampling, typically 12 for monthly or 252 for trading day count"""
    # convert annualized financing rate into appropriate value for provided periodicity
    # risk_free_rate will be converted appropriately in respective Sortino and RMDD calcs
    financing_rate = (1 + financing_rate)**(1 / periodicity) - 1

    #Calculate Replacement Portfolio Sortino Ratio
    replace_port_sortino = sortino_ratio(replace_port, risk_free = risk_free_rate, periodicity = periodicity)

    #Calculate Replacement Portfolio Return to Max Drawdown
    replace_port_return_max_drawdown = return_max_drawdown_ratio(
                                                    replace_port,
                                                    risk_free = risk_free_rate,
                                                    periodicity = periodicity)

    #Calculate New Portfolio Sortino Ratio
    new_port = (new_asset - financing_rate) * weight_asset + replace_port * weight_replace_port
    new_port_sortino = sortino_ratio(new_port, risk_free = risk_free_rate, periodicity = periodicity)

    #Calculate New Portfolio Return to Max Drawdown
    new_port_return_max_drawdown = return_max_drawdown_ratio(new_port, risk_free = risk_free_rate, periodicity = periodicity)

    #Final CWARP calculation
    CWARP = (((new_port_return_max_drawdown / replace_port_return_max_drawdown) * 
              (new_port_sortino / replace_port_sortino)) ** (1/2) - 1) * 100

    return CWARP

In [74]:
ticker_string = "qqq, lqd, hyg, tlt, ief, shy, gld, slv, efa, eem, iyr, xle, xlk, xlf"
ticker_list = ticker_string.replace(' ','').split(',')
risk_free_rate = 0.0
weight_asset = 0.25
financing_rate = 0.00
weight_replace_port = 1.00

weight_replacement_portfolio_stock = 0.6
weight_replacement_portfolio_bond  = 0.4
ticker_replacement_portfolio_stock = 'spy'
ticker_replacement_portfolio_bond  = 'ief'
start_date = '2008-01-01'
end_date = '2020-12-31'

df_stock = retrieve_yahoo_data(ticker_replacement_portfolio_stock, start_date, end_date)
df_bond = retrieve_yahoo_data(ticker_replacement_portfolio_bond, start_date, end_date)
replacement_portfolio = ((weight_replacement_portfolio_stock * df_stock) + 
                        (weight_replacement_portfolio_bond * df_bond))
risk_ret_df = pd.DataFrame(
                index = ['Start_Date','End_Date','CWARP','+Sortino','+Ret_To_MaxDD','Sharpe','Sortino','Max_DD'],
                columns = ticker_list)
new_risk_ret_df = pd.DataFrame(
                    index = ['Return','Vol','Sharpe','Sortino','Max_DD','Ret_To_MaxDD',f'CWARP_{round(100*weight_asset)}%_asset'],
                    columns = ticker_list)

for i in range(0, len(ticker_list)):
    ticker_data = retrieve_yahoo_data(ticker_list[i])
    win_above_replacement_port_df = win_above_replacement_portfolio(
                                    new_asset = ticker_data, 
                                    replace_port = replacement_portfolio,
                                    risk_free_rate = risk_free_rate,
                                    financing_rate = financing_rate,
                                    weight_asset = weight_asset,
                                    weight_replacement_port = weight_replacement_port,
                                    periodicity = 252)
    print(f"Win above replace port for {ticker_list[i]} is {win_above_replace_port_df}")



Ticker is spy
Ticker is ief
Ticker is qqq
Win above replace port for qqq is 4.496153369144373
Ticker is lqd
Win above replace port for lqd is 10.190490893093052
Ticker is hyg
Win above replace port for hyg is -4.241930977628671
Ticker is tlt
Win above replace port for tlt is 33.506809092918274
Ticker is ief
Win above replace port for ief is 19.795654264033224
Ticker is shy
Win above replace port for shy is 5.429839525449287
Ticker is gld
Win above replace port for gld is 14.578977196150046
Ticker is slv
Win above replace port for slv is -6.234214897577961
Ticker is efa
Win above replace port for efa is -25.288025580943106
Ticker is eem
Win above replace port for eem is -24.903872250327332
Ticker is iyr
Win above replace port for iyr is -19.492565449584333
Ticker is xle
Win above replace port for xle is -34.50029680080826
Ticker is xlk
Win above replace port for xlk is 2.1784113892422408
Ticker is xlf
Win above replace port for xlf is -27.565343491675932


Date
2008-01-02         NaN
2008-01-03    0.000532
2008-01-04   -0.013657
2008-01-07    0.000218
2008-01-08   -0.008827
dtype: float64

Date
2008-01-02         NaN
2008-01-03   -0.000483
2008-01-04   -0.024507
2008-01-07   -0.000849
2008-01-08   -0.016149
Name: spy, dtype: float64

Date
2008-01-02         NaN
2008-01-03    0.002054
2008-01-04    0.002618
2008-01-07    0.001817
2008-01-08    0.002154
Name: ief, dtype: float64